## Custom Factors

当我们首先查看因子时，我们探索了一系列内置因子。通常，所需的计算不一定全部包含在内置因子内。`Pipeline API`最强大的功能之一是它允许我们定义我们自己的自定义因子。当所需计算内置函数不存在时，我们可以定义一个自定义因子。

从概念上讲，自定义因子与内置银子等同。它接受`inputs`、`window_length`和`mask`作为构造函数参数，并且每天返回一个`Factor`对象。

我们以一个不存在的计算为例：标准偏差。要创建计算尾随窗口上标准差的因子，我们可以子类化`CustomFactor`并实现一个计算方法，其签名为：

+ *`inputs`是M×N`numpy`数组，其中M是窗口长度，N是证券数量（除非提供掩码，通常约为全部股票数量，大A约3500）。`inputs`是尾随数据窗口。请注意，银子输入列表中提供的每个`BoundColumn`都有一个M×N数组。 每个数组的数据类型将是相应的`BoundColumn`的`dtype`。
+ `out`是一个长度为N的空数组。out将是我们每天定制因子的输出。计算的任务就是将输出值写入到`out`中。
+ `asset_ids`是一个长度为N的整数数组，其中包含与我们的*`inputs`中的列相对应的证券ID
+ `today`是一个`pandas Timestamp`，表示要计算的那一天

其中，*`inputs`和`out`是最常用的。

已添加到管道中的`CustomFactor`实例将每天调用其计算方法。例如，我们定义一个自定义因子，计算过去5天收盘价的标准偏差。首先，让我们将`CustomFactor`和`numpy`添加到我们的导入语句中。

In [1]:
from zipline.pipeline import CustomFactor, Pipeline
import numpy

接下来，定义自定义因子，以使用`numpy.nanstd`计算尾随窗口上的标准偏差：

In [2]:
class StdDev(CustomFactor):
    def compute(self, today, asset_ids, out, values):
        # Calculates the column-wise standard deviation, ignoring NaNs
        out[:] = numpy.nanstd(values, axis=0)

In [3]:
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.research import run_pipeline

In [4]:
def make_pipeline():
    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(columns={'std_dev': std_dev})

最后，让我们在`make_pipeline()`中实例化我们的因子：

当此管道运行时，`StdDev.compute()`将每天通过数据调用，如下所示：

+ values:一个M×N的`numpy`数组，其中M是5（window_length），N是〜4000（当天数据库中证券的数量）
+ out：一个长度为N（〜4000）的空数组。在这个例子中，计算的工作是使用5天收盘价标准差的数组来填充。

In [5]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
result.head()

/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


std_dev
2015-05-05 00:00:00+00:00 平安银行(000001)  0.110707
                          万 科Ａ(000002)  0.203234
                          国农科技(000004)  0.556072
                          世纪星源(000005)  0.000000
                          深振业Ａ(000006)  0.275899

## Default Inputs

在编写自定义因子时，我们可以在我们的`CustomFactor`子类中设置默认`inputs`和`window_length`。例如，定义`TenDayMeanDifference`自定义因子，以使用`numpy.nanmean`计算尾随窗口上两个数据列之间的平均差异。 我们将默认`inputs`设置为`[USEquityPricing.close，USEquityPricing.open]`并将默认`window_length`设置为10：

In [6]:
class TenDayMeanDifference(CustomFactor):
    # Default inputs.
    inputs = [USEquityPricing.close, USEquityPricing.open]
    window_length = 10
    def compute(self, today, asset_ids, out, close, open):
        # Calculates the column-wise mean difference, ignoring NaNs
        out[:] = numpy.nanmean(close - open, axis=0)

请记住，在这种情况下，`close`和`open`每个都是10 x〜3000的`2D numpy`阵列。

现在，如果我们在不提供任何参数的情况下调用`TenDayMeanDifference`，它将使用默认值。

In [7]:
# Computes the 10-day mean difference between the daily open and close prices.
close_open_diff = TenDayMeanDifference()

默认值可以通过在构造函数调用中指定参数来手动覆盖。

In [8]:
# Computes the 10-day mean difference between the daily high and low prices.
high_low_diff = TenDayMeanDifference(inputs=[USEquityPricing.high, USEquityPricing.low])

## Further Example

让我们举另一个例子，我们建立一个动量自定义因子并用它来创建一个过滤器。然后，我们将使用该过滤器作为我们管线上的`screen`。

首先定义一个动量因子，以n日前收盘价除以最近收盘价，其中n是window_length。

In [9]:
class Momentum(CustomFactor):
    # Default inputs
    inputs = [USEquityPricing.close]

    # Compute momentum
    def compute(self, today, assets, out, close):
        out[:] = close[-1] / close[0]

现在，实例化动量因子两次，来创建10天的动量因子和20天的动量因子。我们还要创建一个`positive_momentum`过滤器，当证券10日因子为正且20日因子为正时，返回True。

In [10]:
ten_day_momentum = Momentum(window_length=10)
twenty_day_momentum = Momentum(window_length=20)

positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

接下来，将动量因子和`positive_momentum`过滤器添加到`make_pipeline`。设置`positive_momentum`作为我们管道的`screen`。

In [11]:
def make_pipeline():

    ten_day_momentum = Momentum(window_length=10)
    twenty_day_momentum = Momentum(window_length=20)

    positive_momentum = ((ten_day_momentum > 1) & (twenty_day_momentum > 1))

    std_dev = StdDev(inputs=[USEquityPricing.close], window_length=5)

    return Pipeline(
        columns={
            'std_dev': std_dev,
            'ten_day_momentum': ten_day_momentum,
            'twenty_day_momentum': twenty_day_momentum
        },
        screen=positive_momentum)

运行这条管道可以输出标准偏差和每个动量计算中的10天和20天正向动量。

In [12]:
result = run_pipeline(make_pipeline(), '2018-04-05', '2018-05-15')
result.head()

std_dev  ten_day_momentum  \
2018-04-10 00:00:00+00:00 沙河股份(000014)  0.317238          1.067873   
                          深康佳Ａ(000016)  0.075895          1.051699   
                          深深宝Ａ(000019)  0.488000          1.100412   
                          飞亚达Ａ(000026)  0.236440          1.055126   
                          国药一致(000028)  0.749896          1.012248   

                                        twenty_day_momentum  
2018-04-10 00:00:00+00:00 沙河股份(000014)             1.033275  
                          深康佳Ａ(000016)             1.115987  
                          深深宝Ａ(000019)             1.100412  
                          飞亚达Ａ(000026)             1.038059  
                          国药一致(000028)             1.034997

自定义因子允许我们在流水线中定义自定义计算。 它们通常是在合作伙伴数据集或多个数据列上执行计算的最佳方式。`CustomFactors`的完整文档可以在[这里](https://www.quantopian.com/help#custom-factors)找到。

在下一课中，我们将使用迄今为止学到的一切来为算法创建一个流水线。